# Algoritmo para separar os dados do movimento desejado, utilizando os dados do vídeo com os dados de MoCap que foram sincronizados

--> Bibliotecas necessárias para esta etapa

In [3]:
import numpy as np
import cv2
import pandas as pd
import readmat
from sources_cv2 import defineStartEnd, count_frames

from utils import *
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Escolha do voluntário

In [4]:
# File location

v3d_filename = r'C:\Users\victo\OneDrive - unb.br\Documentos\Faculdade\PIBIC\Validation-of-CoM-trajectory-Estimation-using-neural-network\Estimativa do CoM utilizando sistema BlazePose e MoCap\F_v3d_Subject_15.mat' 

# Escolher a chave

In [5]:
# Reading MoVi files as a Python dictionary
sample = readmat.mat2dict(v3d_filename)
sample.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Subject_15_F'])

In [8]:
# First move sample
x = sample['Subject_15_F']
move = x['move']
move.keys()

dict_keys(['description', 'markerName', 'markerType', 'markerDescription', 'markerSide', 'markerMatch', 'segmentName', 'segmentType', 'segmentDescription', 'segmentSide', 'segmentMatch', 'segmentParent', 'physicalMarkers', 'virtualMarkers', 'lcData', 'flags30', 'flags120', 'motions_list', 'markerGaps', 'virtualMarkerParent', 'markerLocation', 'virtualMarkerLocation', 'jointsAffine_v3d', 'jointsTranslation_v3d', 'jointsExpMaps_v3d', 'jointsGaps_v3d'])

--> Carregue os dados do código anterior que calcula o centro de massa do corpo todo para todos os frames.

In [9]:
array = np.load('Whole body CoM data.npy')

--> Criar o dataframe

In [10]:
Data = array
name_description = move['motions_list']
data_complete = pd.DataFrame(Data,
                           columns = ['X', 'Y'])
data_complete

,X,Y
0,18.929139,986.508717
1,18.938636,986.511618
2,18.950765,986.516038
3,19.017671,986.494681
4,19.000997,986.499087
...,...,...
17841,46.894255,939.087568
17842,46.896500,939.094325
17843,46.874120,939.095647
17844,46.877768,939.117538


# Separando apenas os dados do movimento de caminhar de um lado ao outro

In [11]:
video_path = r'C:\Users\victo\OneDrive - unb.br\Documentos\Faculdade\PIBIC\Validation-of-CoM-trajectory-Estimation-using-neural-network\Estimativa do CoM utilizando sistema BlazePose e MoCap\F_PG1_Subject_15_L.avi'

In [12]:
# Frequencia dos dados

video = cv2.VideoCapture(video_path)
fps = video.get(cv2.CAP_PROP_FPS)
n_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

freq_MoCap = 120

# Escolher o tempo de início aproximado para o movimento de andar de um lado ao outro

--> Estabeleça o minuto e segundos exatos do início do trecho do vídeo que você busca.

In [13]:
# alterar o minuto do vídeo para o movimento
min_init_video = 1
# alterar o segundo do vídeo para o movimento 
seg_init_video = 55

aprox_init_video = int(((min_init_video * 60) + seg_init_video) * fps)

--> Estabeleça manualmente o trecho do vídeo que deseja coletar. Aperte a tecla D repetidas vezes para os frames do vídeo irem para frente e A para recuarem. 

--> Após chegar no frame de início que você deseja, aperte Enter e após chegar no frame final, aperte Enter novamente.

In [14]:
start_end = defineStartEnd(video_path, n_frames, aprox_init_video)
#start_end = [0, 3504]

[3466, 0]
[3466, 3502]


--> Rode o bloco de código seguinte para converter para a frequência de dados do sistema mocap (120 Hz) e cria o arquivo com esses dados separados.

In [15]:
# frame de início e final do movimento
frame_start = start_end[0]
frame_end = start_end[1]

# converter o frame do vídeo do início do movimento para o dado de Mocap correspondente 
temp_start = frame_start / fps
MOCAP_start = int(temp_start * freq_MoCap)

# converter o frame do vídeo do final do movimento para o dado de Mocap correspondente 
temp_end = frame_end / fps
MOCAP_end = int(temp_end * freq_MoCap)

# criando os dados apenas com as informações 
data_gait = data_complete.iloc[MOCAP_start:MOCAP_end + 1,:]
len(data_gait)
data_gait

,X,Y
13864,977.471806,912.392612
13865,968.700176,911.827323
13866,959.899099,911.299103
13867,950.833143,911.069982
13868,941.732116,911.147434
...,...,...
14004,-312.190904,916.706809
14005,-321.059939,915.501908
14006,-330.022761,914.450112
14007,-339.194238,913.471518


In [16]:
np.save('MoCap CoM gait', data_gait)